# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# import xgboost as xgb
from sklearn.svm import SVC

In [2]:
# 데이터 표준화 - 표준화된 데이터르 svc 적용하는데.. 커널별로 적용
# 이러한 과정을 파이프라인에 넣어서 한번에 처리한다.
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import svm

# Load Data

In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [4]:
# train.head()
# train.describe()

# Data Preprocessing

In [5]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [6]:
# train.dtypes

In [7]:
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

In [8]:
# X_encoded 범주형 데이터는 순서가 있는 라벨링으로 변환한 학습용 데이터
X_encoded.head()

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,137.0,42.0,24.0,0.0,47.0,141.0,114.0,124.0,187.0,102.0,...,161.0,67.0,32.0,26.0,121.0,60.0,51.0,51.0,34.0,40.0
1,137.0,42.0,24.0,0.0,47.0,141.0,114.0,124.0,187.0,102.0,...,161.0,67.0,32.0,26.0,121.0,60.0,51.0,51.0,34.0,40.0
2,104.0,42.0,24.0,0.0,47.0,141.0,114.0,124.0,187.0,102.0,...,161.0,67.0,32.0,26.0,121.0,60.0,51.0,51.0,34.0,40.0
3,137.0,42.0,24.0,0.0,47.0,141.0,114.0,124.0,187.0,102.0,...,161.0,67.0,32.0,26.0,121.0,60.0,51.0,51.0,34.0,40.0
4,137.0,42.0,24.0,0.0,47.0,141.0,114.0,124.0,187.0,102.0,...,161.0,67.0,32.0,26.0,121.0,60.0,51.0,51.0,34.0,40.0


# Model Define and Train

In [9]:
# model = xgb.XGBClassifier(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     use_label_encoder=False,
#     eval_metric='mlogloss' 
# )

# model = SVC()
# SVC는 이진분류에 특화되어 있음
# 다중분류는 OVR


In [10]:
# 평가용 함수
def print_score(clf, x_train,y_train,x_test,y_test, train=True):
    if train:
        pred = clf.predict(x_train)
        clf_report = pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result : \n ==============================================")
        print(f"Accuracy score : {accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------------------------------------")
        print(f"Calssfication Report:\n{clf_report}")
        print('---------------------------------------------------------------')
        print(f'Confusion Matrix: \n{confusion_matrix(y_train,pred)}\n')
    else:
        pred = clf.predict(x_test)
        clf_report = pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result : \n ==============================================")
        print(f"Accuracy score : {accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------------------------------------")
        print(f"Calssfication Report:\n{clf_report}")
        print('---------------------------------------------------------------')
        print(f'Confusion Matrix: \n{confusion_matrix(y_test,pred)}\n')

In [11]:
# x_train,x_test,y_train,y_test = train_test_split(X_encoded,y_subclass,stratify=y_subclass,random_state=42)

# pipeline = Pipeline([
#     ('std_scaler',StandardScaler()),
#     ('logistic',LogisticRegression(max_iter=10000,C=0.01,penalty='l2',solver = "newton-cg" ))
# ])
# pipeline.fit(x_train,y_train)

# # model = SVC()
# # model = LogisticRegression(max_iter=10000)
# # model.fit(X_encoded, y_subclass)

# print_score(pipeline,x_train,y_train,x_test,y_test,train=True)


In [12]:
# print_score(pipeline,x_train,y_train,x_test,y_test,train=False)

In [13]:
x_train,x_test,y_train,y_test = train_test_split(X_encoded,y_subclass,stratify=y_subclass,random_state=42)

pipeline = Pipeline([
    ('std_scaler',StandardScaler()),
    ('logistic',LogisticRegression())
])
param_grid = {
    'logistic__C' : [0.1, 10],
    'logistic__penalty' : ['l1','l2'],
    'logistic__max_iter' : [10000],
    # 'logistic__class_weight' : ['balanced', None],
    # 'logistic__multi_class' : ['auto', 'ovr', 'nultinomial'],
    # 'logistic__tol' : [1e-4, 1e-3, 1e-2],
    'logistic__solver' : ['newton-cg','lbfgs'] # 'saga'
}

gridcv = GridSearchCV(pipeline,param_grid=param_grid,cv=3,verbose=2)
# 하이퍼 파라이터 적용
gridcv.fit(x_train,y_train)
print(f'best parameters : {gridcv.best_params_}')
model = gridcv.best_estimator_


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=newton-cg; total time=   0.8s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=newton-cg; total time=   0.7s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=newton-cg; total time=   0.7s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs; total time=   0.7s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs; total time=   0.8s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l1, logistic__solver=lbfgs; total time=   0.6s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cg; total time=  26.9s
[CV] END logistic__C=0.1, logistic__max_iter=10000, logistic__penalty=l2, logistic__solver=newton-cg; tot

C:\Users\greatsangho\AppData\Local\anaconda3\envs\mypjt001\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\greatsangho\AppData\Local\anaconda3\envs\mypjt001\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\greatsangho\AppData\Local\anaconda3\envs\mypjt001\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\g

best parameters : {'logistic__C': 0.1, 'logistic__max_iter': 10000, 'logistic__penalty': 'l2', 'logistic__solver': 'newton-cg'}


In [14]:
print_score(model,x_train,y_train,x_test,y_test,train=True)
print_score(model,x_train,y_train,x_test,y_test,train=False)

Train Result : 
Accuracy score : 92.65%
---------------------------------------------------------------
Calssfication Report:
              0     1           2           3      4     5           6  \
precision   1.0   1.0    0.994845    1.000000    1.0   1.0    0.858790   
recall      1.0   1.0    0.983022    0.991379    1.0   1.0    0.861272   
f1-score    1.0   1.0    0.988898    0.995671    1.0   1.0    0.860029   
support    54.0  78.0  589.000000  116.000000  167.0  29.0  346.000000   

                    7           8           9  ...          19     20  \
precision    1.000000    0.791457    0.716814  ...    1.000000    1.0   
recall       0.994012    0.816062    0.648000  ...    0.959459    1.0   
f1-score     0.996997    0.803571    0.680672  ...    0.979310    1.0   
support    167.000000  386.000000  250.000000  ...  148.000000  207.0   

                   21         22         23         24     25  accuracy  \
precision    1.000000   1.000000    0.72973   1.000000    1.0 

# Inference

In [15]:
test_X = test.drop(columns=['ID'])
X_encoded = test_X.copy()
X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns]) # 학습 데이터와 동일하게 맞춤

In [16]:
# 테스트 데이터 예측
predictions = model.predict(X_encoded)

In [17]:
predictions

array([21, 21, 17, ...,  1,  8,  6])

In [18]:
original_labels = le_subclass.inverse_transform(predictions)
original_labels

array(['STES', 'STES', 'PCPG', ..., 'BLCA', 'KIPAN', 'GBMLGG'],
      dtype=object)

# Submisson

In [19]:
submisson = pd.read_csv("./sample_submission.csv")
submisson

,ID,SUBCLASS
0,TEST_0000,LGG
1,TEST_0001,LGG
2,TEST_0002,LGG
3,TEST_0003,LGG
4,TEST_0004,LGG
...,...,...
2541,TEST_2541,LGG
2542,TEST_2542,LGG
2543,TEST_2543,LGG
2544,TEST_2544,LGG


In [20]:
submisson["SUBCLASS"] = original_labels

In [21]:
submisson.to_csv('./baseline_submission.csv', encoding='UTF-8-sig', index=False)

In [22]:
# https://scikit-learn.org/stable/supervised_learning.html
# sklearn 공식 사이트